In [1]:
!pip install langchain-groq
!pip install crewai
!pip install duckduckgo-search
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from crewai import Agent,Task,Crew,Process
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [3]:
llm = ChatGroq(temperature=0, groq_api_key="", model_name="mixtral-8x7b-32768")

researcher=Agent(
    role="Senior Researcher",
    goal="uncover top 3 rending news in {topic}",
    verbose=True,
    memory=True,
    backstory=(
        """
    As a research assistant dedicated to uncovering the most impactful trends,
    you're propelled by a relentless curiosity and a commitment to innovation.
    Your role involves delving deep into the latest developments
    across various sectors to identify and analyze
    the top trending news within any given field.
    This pursuit not only satisfies your thirst for knowledge
    but also enables you to contribute valuable insights that could
    potentially reshape understandings and expectations on a global scale
    """
    ),
    tools=[search_tool],
    allow_delegation=True,
    llm=llm
)

blog_writer=Agent(
    role="Expert Writer",
    goal="write compelling contents about {topic}",
    verbose=True,
    memory=True,
    backstory=(
    """
     Armed with the knack for distilling complex subjects into digestible,
    compelling stories, you, as a blog writer, masterfully weave narratives
    that both enlighten and engage your audience. Your writing illuminates fresh
    insights and discoveries, making them approachable for everyone. Through your craft,
    you bring to the forefront the essence of new developments across various topics,
    making the intricate world of news a fascinating journey for your readers.
    """
    ),
    tools=[search_tool],
    allow_delegation=False,
    llm=llm
)

research_task=Task(
    description=(
    """
    Identify the next big trend in {topic}.
    Focus on identifying pros and cons and the overall narrative.
    Your final report should clearly articulate the key points
    its market opportunities, and potential risks.
    """
    ),
    expected_output="A comprehensive 3 paragraphs long report on the {topic}",
    tools=[search_tool],
    agent=researcher
)

write_task=Task(
    description=(
    """
    Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive.
    """
    ),
    expected_output="A 4 paragraph article on {topic} advancements formatted as markdown",
    tools=[search_tool],
    agent=blog_writer,
    aync_execution=False,
    output_file="blog-post.md"
)
crew=Crew(
    agents=[researcher,blog_writer],
    tasks=[research_task,write_task],
    process=Process.sequential
)

result=crew.kickoff(inputs={"topic":"Latest smart phones"})



> Entering new CrewAgentExecutor chain...
To identify the next big trend in latest smartphones, I need to gather information about the latest developments and innovations in this field. I will start by conducting a broad search to get an overview of the current trends.

Action: duckduckgo_search
Action Input: {"query": "latest trends in smartphones 2023"}
 

In 2023, one could argue that it was a steady course in the world of phones. Apple released its latest iPhone 15 series, Samsung set the benchmark high for the best Android phones with the Galaxy ... Fall 2023: An avalanche of new smartphone options Fall 2023 brought some of the most impressive phones in the year as a whole. Let's start with the Made by Google event, where the Pixel 8 and ... Samsung Galaxy Z Flip 5 (256GB) $ 800 $ 999 20 % off. Samsung's Z Flip 5 is its latest 6.7-inch flip phone that folds in half, now sporting a larger front cover display and a new hinge design ... Best Pixel. 4. Google Pixel 8 Pro. View at Am

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01hqm6p29vex4aajjv44d7d7gf` on tokens per minute (TPM): Limit 2500, Used 2688, Requested ~2242. Please try again in 58.336s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}